In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', accent_classification=True)
data.generate_data('letters_accent_classification_train',
                   'letters_accent_classification_test',
                   'letters_accent_classification_validate', force_override=False)

LOADING DATA...
LOAD SUCCESSFUL!


In [5]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [102]:
test2 = next(gen2)

In [129]:
test2 = next(gen2)
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
print(len(test2[0]))
print(len(test2))
for el in test2[0][0]:
#     print(el)
    print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

BBB
10
(16, 23, 36)
(16, 150)
2
2
invitagen
iremirp
iksneko
jnavolimop
mišjentsačilevjan
mišjentsačilevjan
hišjenbordopjan
hišjenbordopjan
enitesjavd
amonobarab
enejlugo
amtsonlanoiseforp
anavokilboen
anavokilboen
ebžalot
invejark


In [6]:
def count_vowels(content, accented_vowels):
    num_all_vowels = 0
    for el in content:
        for m in list(el[3]):
            if m in accented_vowels:
                num_all_vowels += 1
    return num_all_vowels

count_vowels(content, accented_vowels)

592886

In [10]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.decode_y(data.y_train[pos]))
data.y_train[pos]

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[2, 5]


array([ 0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.])

In [4]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [6]:
# num_examples = count_vowels(content, accented_vowels) # training set size
num_examples = len(data.x_train)
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20


# num_examples = len(data.x_train) # training set size
# nn_output_dim = 11
# nn_hdim = 516
# batch_size = 16
# actual_epoch = 1
# num_fake_epoch = 20



conv_input_shape=(23, 36)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [7]:
y_array_train = np.asarray(data.y_train)
accentuation_length_train = (y_array_train > 0).sum()
y_array_test = np.asarray(data.y_test)
accentuation_length_test = (y_array_test > 0).sum()
history = model.fit_generator(data.generator('train', batch_size), 
                              accentuation_length_train/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch)

Epoch 1/400
1625/1624 [==============================] - 32s - loss: 0.2328 - actual_accuracy: 0.0629    
Epoch 2/400
1625/1624 [==============================] - 34s - loss: 0.1630 - actual_accuracy: 0.3024    
Epoch 3/400
1625/1624 [==============================] - 35s - loss: 0.1252 - actual_accuracy: 0.5025    
Epoch 4/400
1625/1624 [==============================] - 34s - loss: 0.0986 - actual_accuracy: 0.6400    - ETA: 1s - loss: 
Epoch 5/400
1625/1624 [==============================] - 34s - loss: 0.0806 - actual_accuracy: 0.7141    
Epoch 6/400
1625/1624 [==============================] - 35s - loss: 0.0664 - actual_accuracy: 0.7698    
Epoch 7/400
1625/1624 [==============================] - 34s - loss: 0.0558 - actual_accuracy: 0.8193    
Epoch 8/400
1625/1624 [==============================] - 35s - loss: 0.0487 - actual_accuracy: 0.8484    
Epoch 9/400
1625/1624 [==============================] - 35s - loss: 0.0429 - actual_accuracy: 0.8701    
Epoch 10/400
1625/1624 [====

1625/1624 [==============================] - 26s - loss: 0.0061 - actual_accuracy: 0.9824    
Epoch 77/400
1625/1624 [==============================] - 28s - loss: 0.0057 - actual_accuracy: 0.9836    
Epoch 78/400
1625/1624 [==============================] - 27s - loss: 0.0072 - actual_accuracy: 0.9800    
Epoch 79/400
1625/1624 [==============================] - 27s - loss: 0.0050 - actual_accuracy: 0.9854    
Epoch 80/400
1625/1624 [==============================] - 29s - loss: 0.0041 - actual_accuracy: 0.9889    
Epoch 81/400
1625/1624 [==============================] - 30s - loss: 0.0071 - actual_accuracy: 0.9803    
Epoch 82/400
1625/1624 [==============================] - 30s - loss: 0.0060 - actual_accuracy: 0.9827    
Epoch 83/400
1625/1624 [==============================] - 29s - loss: 0.0057 - actual_accuracy: 0.9828    
Epoch 84/400
1625/1624 [==============================] - 26s - loss: 0.0062 - actual_accuracy: 0.9835    
Epoch 85/400
1625/1624 [==========================

1625/1624 [==============================] - 34s - loss: 0.0033 - actual_accuracy: 0.9914    
Epoch 151/400
1625/1624 [==============================] - 28s - loss: 0.0035 - actual_accuracy: 0.9892    
Epoch 152/400
1625/1624 [==============================] - 28s - loss: 0.0037 - actual_accuracy: 0.9887    
Epoch 153/400
1625/1624 [==============================] - 29s - loss: 0.0038 - actual_accuracy: 0.9900    
Epoch 154/400
1625/1624 [==============================] - 28s - loss: 0.0037 - actual_accuracy: 0.9901    
Epoch 155/400
1625/1624 [==============================] - 27s - loss: 0.0033 - actual_accuracy: 0.9910    
Epoch 156/400
1625/1624 [==============================] - 28s - loss: 0.0035 - actual_accuracy: 0.9907    - ETA: 1s - loss: 0
Epoch 157/400
1625/1624 [==============================] - 28s - loss: 0.0034 - actual_accuracy: 0.9906    
Epoch 158/400
1625/1624 [==============================] - 28s - loss: 0.0037 - actual_accuracy: 0.9900    
Epoch 159/400
1625/1624

1625/1624 [==============================] - 29s - loss: 0.0028 - actual_accuracy: 0.9930    
Epoch 226/400
1625/1624 [==============================] - 29s - loss: 0.0029 - actual_accuracy: 0.9923    
Epoch 227/400
1625/1624 [==============================] - 29s - loss: 0.0029 - actual_accuracy: 0.9927    
Epoch 228/400
1625/1624 [==============================] - 29s - loss: 0.0032 - actual_accuracy: 0.9912    
Epoch 229/400
1625/1624 [==============================] - 28s - loss: 0.0029 - actual_accuracy: 0.9926    
Epoch 230/400
1625/1624 [==============================] - 29s - loss: 0.0028 - actual_accuracy: 0.9927    
Epoch 231/400
1625/1624 [==============================] - 30s - loss: 0.0022 - actual_accuracy: 0.9937    
Epoch 232/400
1625/1624 [==============================] - 29s - loss: 0.0031 - actual_accuracy: 0.9921    
Epoch 233/400
1625/1624 [==============================] - 30s - loss: 0.0029 - actual_accuracy: 0.9927    
Epoch 234/400
1625/1624 [=================

1625/1624 [==============================] - 32s - loss: 0.0021 - actual_accuracy: 0.9947    
Epoch 301/400
1625/1624 [==============================] - 31s - loss: 0.0025 - actual_accuracy: 0.9937    
Epoch 302/400
1625/1624 [==============================] - 32s - loss: 0.0026 - actual_accuracy: 0.9934    
Epoch 303/400
1625/1624 [==============================] - 32s - loss: 0.0024 - actual_accuracy: 0.9934    
Epoch 304/400
1625/1624 [==============================] - 32s - loss: 0.0022 - actual_accuracy: 0.9935    
Epoch 305/400
1625/1624 [==============================] - 34s - loss: 0.0023 - actual_accuracy: 0.9939    
Epoch 306/400
1625/1624 [==============================] - 46s - loss: 0.0021 - actual_accuracy: 0.9944    
Epoch 307/400
1625/1624 [==============================] - 43s - loss: 0.0025 - actual_accuracy: 0.9932    - ETA: 0s - loss: 0.0025 - actual_accur
Epoch 308/400
1625/1624 [==============================] - 41s - loss: 0.0026 - actual_accuracy: 0.9930    - ET

1625/1624 [==============================] - 38s - loss: 0.0024 - actual_accuracy: 0.9932    
Epoch 374/400
1625/1624 [==============================] - 29s - loss: 0.0019 - actual_accuracy: 0.9946    
Epoch 375/400
1625/1624 [==============================] - 36s - loss: 0.0023 - actual_accuracy: 0.9947    
Epoch 376/400
1625/1624 [==============================] - 40s - loss: 0.0023 - actual_accuracy: 0.9940    
Epoch 377/400
1625/1624 [==============================] - 40s - loss: 0.0020 - actual_accuracy: 0.9942    
Epoch 378/400
1625/1624 [==============================] - 42s - loss: 0.0026 - actual_accuracy: 0.9931    
Epoch 379/400
1625/1624 [==============================] - 41s - loss: 0.0020 - actual_accuracy: 0.9942    
Epoch 380/400
1625/1624 [==============================] - 43s - loss: 0.0016 - actual_accuracy: 0.9957    - ETA: 0s - loss: 0.0016 - actual_accuracy:
Epoch 381/400
1625/1624 [==============================] - 43s - loss: 0.0026 - actual_accuracy: 0.9937    

In [6]:
name = '20_test_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [9]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()